In [ ]:
import RPi.GPIO as GPIO
import time
from gpiozero import Buzzer, MotionSensor
from threading import Thread

# --- Configuración del Buzzer y Sensor PIR ---
buzzer_zumbador = Buzzer(24)  # Conexión del buzzer al pin GPIO 24
pir = MotionSensor(23)  # Conexión del sensor PIR al pin GPIO 23

# --- Configuración del Teclado Matricial ---
# Pines del teclado
R1 = 29
R2 = 31
R3 = 33
R4 = 35

C1 = 32
C2 = 36
C3 = 38
C4 = 40

# Configuración GPIO para el teclado
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BOARD)

GPIO.setup(R1, GPIO.OUT)
GPIO.setup(R2, GPIO.OUT)
GPIO.setup(R3, GPIO.OUT)
GPIO.setup(R4, GPIO.OUT)

GPIO.setup(C1, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(C2, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(C3, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)
GPIO.setup(C4, GPIO.IN, pull_up_down=GPIO.PUD_DOWN)

# Contraseña predefinida
PASSWORD = "123A"
input_sequence = ""  # Cadena para registrar la entrada del usuario
motion_detector_enabled = True  # El detector de movimiento comienza activado
message_displayed = False  # Bandera para evitar mostrar mensajes repetidos

# --- Función para detectar teclas ---
def detectKeyPress(row, character):
    global input_sequence
    GPIO.output(row, GPIO.HIGH)  # Activa la fila

    for col, key in zip([C1, C2, C3, C4], character):
        if GPIO.input(col) == 1:  # Si la tecla está presionada
            print(f"Tecla presionada: {key}")
            input_sequence += key  # Agrega la tecla presionada a la secuencia
            time.sleep(0.5)  # Pequeña pausa para evitar múltiples registros

    GPIO.output(row, GPIO.LOW)  # Desactiva la fila

# --- Función para verificar la clave ---
def verify_password(prompt):
    global input_sequence
    print(prompt)

    while True:
        detectKeyPress(R1, ["1", "2", "3", "A"])
        detectKeyPress(R2, ["4", "5", "6", "B"])
        detectKeyPress(R3, ["7", "8", "9", "C"])
        detectKeyPress(R4, ["*", "0", "#", "D"])

        if len(input_sequence) == len(PASSWORD):
            if input_sequence == PASSWORD:
                input_sequence = ""  # Reinicia la entrada
                return True
            else:
                print("Clave incorrecta. Intente de nuevo.")
                input_sequence = ""  # Reinicia la entrada
        time.sleep(0.1)

# --- Función para monitorear el detector de movimiento ---
def monitor_motion():
    global motion_detector_enabled, message_displayed
    while True:
        if motion_detector_enabled:
            print("Monitoreando movimiento...")
            pir.wait_for_motion()
            print("¡Movimiento detectado!")
            buzzer_zumbador.on()
            print("Ingrese la clave de 4 dígitos para desactivar la alarma.")
            message_displayed = False  # Reiniciar la bandera
            time.sleep(0.5)  # Prevenir activaciones continuas
        else:
            buzzer_zumbador.off()  # Asegurar que el buzzer esté apagado
            if not message_displayed:
                print("Ingrese la clave de 4 dígitos para activar la alarma.")
                message_displayed = True  # Evitar mostrar el mensaje repetidamente
            time.sleep(0.5)  # Reducir uso de CPU cuando el detector está desactivado

# --- Ciclo Principal ---
try:
    print("Sistema de alarma iniciado. Detector de movimiento está activado.")

    # Hilo separado para monitorear movimiento
    thread = Thread(target=monitor_motion)
    thread.daemon = True
    thread.start()

    while True:
        if motion_detector_enabled:
            # Solicitar clave para desactivar
            if verify_password("Ingrese la clave de 4 dígitos para desactivar la alarma:"):
                print("Clave correcta alarma desactivada.")
                buzzer_zumbador.off()  # Apagar el buzzer si estaba activo
                motion_detector_enabled = False
        else:
            # Solicitar clave para activar
            if verify_password("Ingrese la clave de 4 dígitos para activar la alarma:"):
                print("Clave correcta.Alarma activada.")
                motion_detector_enabled = True
                message_displayed = False  # Permitir mostrar el mensaje nuevamente

except KeyboardInterrupt:
    print("Programa detenido por el usuario.")
finally:
    GPIO.cleanup()  # Limpia la configuración GPIO al salir

